##  Probit models to forecast binary outcomes such as recessions
<div style="text-align: right"> Fogli Alessandro </div>
<div style="text-align: right"> ID 231273 </div>
<div style="text-align: right"> Project #3 </div>

### Install packages

In [85]:
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import yfinance as yf
import pandas_datareader as pdr
from IPython.display import display, HTML
import datetime as dt
import getFamaFrenchFactors as gff
import quandl
from fredapi import Fred
import config
fred = Fred(api_key= config.fred_api)
QUANDL_KEY = config.quandl_key
quandl.ApiConfig.api_key = QUANDL_KEY

### Data (all quarterly)

Get data of difference in yields between 10 year and 3 months U.S. Treasuries

In [42]:
t10y = fred.get_series('DGS10', observation_start="1962-01-02",observation_end= "2021-12-31",frequency='q')
t3mo = fred.get_series('TB3MS', observation_start="1962-01-02",observation_end= "2021-12-31",frequency='q')
term = t10y-t3mo
term = term.dropna()
term = term.tolist()
#240

Get data of Federal Funds Rate

In [43]:
funds_rate = fred.get_series('FEDFUNDS', observation_start="1962-01-02", observation_end= "2021-12-31" ,frequency='q')
funds_rate = funds_rate.tolist()
#240

Get data of GDP Growth

In [45]:
gdp = fred.get_series('GDP', observation_start="1962-01-02", observation_end= "2021-12-31",frequency='q', units='pch')
gdp = gdp.tolist()
#240 miss first quarter 2022

NBER based Recession Indicators for the United States

In [56]:
nber = fred.get_series('USREC', observation_start="1962-01-02", observation_end= "2021-12-31" ,frequency='q')
nber = nber.astype(int)
nber = nber.tolist()

Load data of S&P500 Futures - Jun 2022 expiring date

In [84]:
sp500_futures = pd.read_csv('S&P 500 Futures Historical Data.csv', index_col = 0)
sp500_futures.index = pd.to_datetime(sp500_futures.index, format= '%b %y') # date index transforming
sp500_futures.index = sp500_futures.index + pd.offsets.MonthEnd()
sp500_futures_rtn = sp500_futures['Change %']
sp500_futures_rtn= sp500_futures_rtn.str.replace('%','') # remove str % from value
sp500_futures_rtn = pd.to_numeric(sp500_futures_rtn, errors='coerce')
sp500_futures_rtn.fillna(0, inplace=True)
sp500_futures_rtn = sp500_futures_rtn.iloc[::-1]
sp500_futures_rtn.drop(sp500_futures_rtn.tail(1).index,inplace=True) # adjusting size to match stocks month

Get data of S&P500

In [81]:
sp500 = quandl.get("MULTPL/SP500_REAL_PRICE_MONTH", start_date='1961-10-01', end_date='2021-12-31', collapse="monthly")
sp500_quarter_rtn = sp500.resample("3M").mean()
sp500_quarter_rtn = sp500_quarter_rtn.pct_change()
sp500_quarter_rtn = sp500_quarter_rtn.dropna()
sp500_quarter_rtn.drop(sp500_quarter_rtn.tail(1).index,inplace=True)
sp500_quarter_rtn = sp500_quarter_rtn.apply(lambda x: x* 100)

### Probit model

A probit regression is a version of the generalized linear model used to model dichotomous outcome variables. It uses the inverse standard normal distribution as a linear combination of the predictors. The binary outcome variable Y is assumed to have a Bernoulli distribution with parameter p (where the success probability is p∈(0,1). Hence, the probit link function is:
$$ probit(Y) = \sum_{k=0}^n \beta_{k} x_{ik} $$

The Probit model assumes that the firm’s probability of recession has a cumulative standard-normal distribution, rather than a logistic distribution. However, by multiplying the results of the logistic distribution by an appropriate coefficient the distribution of the Probit model can be obtained.

In [82]:
rec_data["DATE"] = nber.to_datetime(rec_data["DATE"])
ls_data["DATE"] = term.to_datetime(ls_data["DATE"])
fig, ax = plt.subplots()
ls_data.plot.line(ax=ax, figsize=(8, 5), x='DATE', color="blue")

AttributeError: 'list' object has no attribute 'shape'